In [1]:
import tensorflow as tf
import numpy as np
from msalib import layers
from msalib import network
from msalib import train
from msalib.utils import Dataset
import yaml


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
def target(x):
    """Target function

    Arguments:
        x {numpy array} -- input

    Returns:
        numpy array -- output
    """

    return np.sin(x)


def loss_func(output, label):
    """Loss function (l2)

    Arguments:
        output {tf tensor} -- output from network
        label {tf tensor} -- labels

    Returns:
        tf tensor -- loss
    """

    output = tf.reduce_sum(output, axis=1, keepdims=True)
    return tf.losses.mean_squared_error(
        predictions=output, labels=label)


def generate_datasets(config):
    """Generate synthetic dataset

    Arguments:
        config {dict} -- read from config.yml

    Returns:
        tuple of Dataset objects -- train/test sets
    """

    np.random.seed(config['seed'])
    num_train, num_test = config['num_train'], config['num_test']
    x_train = np.random.uniform(-np.pi, np.pi, (num_train, 1))
    x_test = np.random.uniform(-np.pi, np.pi, (num_test, 1))
    y_train, y_test = target(x_train), target(x_test)
    x_train = np.repeat(x_train, axis=1, repeats=config['num_nodes'])
    x_test = np.repeat(x_test, axis=1, repeats=config['num_nodes'])
    trainset = Dataset(data=x_train, target=y_train)
    testset = Dataset(data=x_test, target=y_test)
    return trainset, testset


def run_toy(config):
    """Run toy problem

    Arguments:
        config {dict} -- read from config.yml
    """

    print('='*100)
    print('Run toy model')
    print('='*100)
    print('Config:')
    for key, value in config.items():
        print('{:20} ({})'.format(key, value))
    print('='*100)

    tf.logging.set_verbosity(
        getattr(tf.logging, config['verbosity']))
    tf.set_random_seed(config['seed'])

    # Generate data
    trainset, testset = generate_datasets(config)

    # Build network
    input = tf.placeholder(
        tf.float32, [None, config['num_nodes']], name='input')
    output = tf.placeholder(tf.float32, [None, 1], name='output')
    net = network.Network(name='msa_net')

    for n in range(config['num_layers']):
        if n == 0:
            net.add(layers.ResidualDense(
                input_shape=(input.shape[1:]),
                units=config['num_nodes'], activation=config['activation'],
                msa_rho=config['rho'], msa_reg=config['reg'],
                kernel_initializer=tf.truncated_normal_initializer(
                    stddev=config['kernel_init']),
                bias_initializer=tf.constant_initializer(config['bias_init']),
                delta=config['delta'], name='residualdense_{}'.format(n)))
        else:
            net.add(layers.ResidualDense(
                units=config['num_nodes'], activation=config['activation'],
                msa_rho=config['rho'], msa_reg=config['reg'],
                kernel_initializer=tf.truncated_normal_initializer(
                    stddev=config['kernel_init']),
                bias_initializer=tf.constant_initializer(config['bias_init']),
                delta=config['delta'], name='residualdense_{}'.format(n)))

    net.msa_compute_x(input)
    net.msa_compute_p(output, loss_func)
    net.summary()
    sess = tf.Session()

    # MSA trainer
    msa_trainer = train.MSATrainer(
        network=net,
        name='MSA_trainer',
        maxiter=config['msa_maxiter'],
        perturb_init=config['msa_perturb_init'])
    msa_trainer.initialize(sess)
    msa_trainer.train(
        session=sess,
        trainset=trainset,
        testset=testset,
        batch_size=config['batch_size'],
        num_epochs=config['num_epochs'],
        buffer_size=config['buffer_size'],
        print_step=config['print_step'])

    # SGD trainer
    sgd_trainer = train.BPTrainer(
        network=net, name='SGD_trainer',
        method='GradientDescentOptimizer',
        args={'learning_rate': config['sgd_lr']})
    sgd_trainer.initialize(sess)
    sgd_trainer.train(
        session=sess,
        trainset=trainset,
        testset=testset,
        batch_size=config['batch_size'],
        num_epochs=config['num_epochs'],
        buffer_size=config['buffer_size'],
        print_step=config['print_step'])

    # Adagrad trainer
    adagrad_trainer = train.BPTrainer(
        network=net, name='Adagrad_trainer',
        method='AdagradOptimizer',
        args={'learning_rate': config['adagrad_lr']})
    adagrad_trainer.initialize(sess)
    adagrad_trainer.train(
        session=sess,
        trainset=trainset,
        testset=testset,
        batch_size=config['batch_size'],
        num_epochs=config['num_epochs'],
        buffer_size=config['buffer_size'],
        print_step=config['print_step'])

    # Adam trainer
    Adam_trainer = train.BPTrainer(
        network=net, name='Adam_trainer',
        method='AdamOptimizer',
        args={'learning_rate': config['adam_lr']})
    Adam_trainer.initialize(sess)
    Adam_trainer.train(
        session=sess,
        trainset=trainset,
        testset=testset,
        batch_size=config['batch_size'],
        num_epochs=config['num_epochs'],
        buffer_size=config['buffer_size'],
        print_step=config['print_step'])


In [3]:
!python3 -u main_toy.py | tee sin.log

^C
Traceback (most recent call last):
  File "main_toy.py", line 5, in <module>
    from msalib import train
  File "/mnt/d/lesenok/Linux/CNN/msalib/train.py", line 3, in <module>
    from msalib.scipyoptimizer import ScipyOptimizer
  File "/mnt/d/lesenok/Linux/CNN/msalib/scipyoptimizer.py", line 5, in <module>
    Base = tf.contrib.opt.ScipyOptimizerInterface
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/lazy_loader.py", line 61, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/lazy_loader.py", line 44, in _load
    module = importlib.import_module(self.__name__)
  File "/usr/lib/python3.7/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/contrib/__init__.py", line 95, in <module>
    from tensorflow.contrib.optimizer_v2 import optimizer_v2_symbols as optimizer_v2
  File "/usr/lo

In [3]:
 if __name__ == "__main__":
    config = yaml.safe_load(open("config.yml"))
    run_toy(config['toy'])
    

Run toy model
Config:
seed                 (1)
verbosity            (WARN)
print_step           (False)
num_train            (100)
num_test             (100)
num_nodes            (5)
num_layers           (20)
rho                  (1.0)
reg                  (0.001)
activation           (tanh)
kernel_init          (0.0)
bias_init            (0.0)
delta                (0.25)
batch_size           (100)
buffer_size          (100)
num_epochs           (500)
sgd_lr               (0.1)
adagrad_lr           (0.1)
adam_lr              (0.005)
msa_maxiter          (10)
msa_perturb_init     (0.005)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
residualdense_0 (ResidualDen (None, 5)                 30        
________________________________________

Epoch: 45
Train loss: 0.11878462880849838
Test loss: 0.12081722170114517
Epoch: 46
Train loss: 0.11566396057605743
Test loss: 0.11742264032363892
Epoch: 47
Train loss: 0.1127704381942749
Test loss: 0.11432130634784698
Epoch: 48
Train loss: 0.1096314862370491
Test loss: 0.11101696640253067
Epoch: 49
Train loss: 0.10689201951026917
Test loss: 0.10819574445486069
Epoch: 50
Train loss: 0.10355933010578156
Test loss: 0.10460850596427917
Epoch: 51
Train loss: 0.101222924888134
Test loss: 0.10218772292137146
Epoch: 52
Train loss: 0.099486343562603
Test loss: 0.1003652811050415
Epoch: 53
Train loss: 0.09693531692028046
Test loss: 0.097777359187603
Epoch: 54
Train loss: 0.09542710334062576
Test loss: 0.09622682631015778
Epoch: 55
Train loss: 0.09394341707229614
Test loss: 0.09467333555221558
Epoch: 56
Train loss: 0.0917004719376564
Test loss: 0.09243670105934143
Epoch: 57
Train loss: 0.09005194157361984
Test loss: 0.09080243110656738
Epoch: 58
Train loss: 0.08838255703449249
Test loss: 0.089161

Epoch: 157
Train loss: 0.04318348318338394
Test loss: 0.043638937175273895
Epoch: 158
Train loss: 0.043049704283475876
Test loss: 0.04349137842655182
Epoch: 159
Train loss: 0.04289362207055092
Test loss: 0.04331818222999573
Epoch: 160
Train loss: 0.04272432252764702
Test loss: 0.043149787932634354
Epoch: 161
Train loss: 0.04255576431751251
Test loss: 0.04297732561826706
Epoch: 162
Train loss: 0.042432110756635666
Test loss: 0.042862337082624435
Epoch: 163
Train loss: 0.04227207973599434
Test loss: 0.04270379990339279
Epoch: 164
Train loss: 0.04211978614330292
Test loss: 0.042541906237602234
Epoch: 165
Train loss: 0.04193400591611862
Test loss: 0.04235941544175148
Epoch: 166
Train loss: 0.04178958386182785
Test loss: 0.042221490293741226
Epoch: 167
Train loss: 0.04166826605796814
Test loss: 0.042111415416002274
Epoch: 168
Train loss: 0.04154011234641075
Test loss: 0.04198412224650383
Epoch: 169
Train loss: 0.04139349237084389
Test loss: 0.041829731315374374
Epoch: 170
Train loss: 0.0412

Epoch: 267
Train loss: 0.032345954328775406
Test loss: 0.0328943207859993
Epoch: 268
Train loss: 0.032289162278175354
Test loss: 0.03282513841986656
Epoch: 269
Train loss: 0.03223578631877899
Test loss: 0.03277904540300369
Epoch: 270
Train loss: 0.03217713162302971
Test loss: 0.03272049501538277
Epoch: 271
Train loss: 0.03212563320994377
Test loss: 0.03266539424657822
Epoch: 272
Train loss: 0.03209228813648224
Test loss: 0.03262634947896004
Epoch: 273
Train loss: 0.03205612301826477
Test loss: 0.03259371966123581
Epoch: 274
Train loss: 0.03200172632932663
Test loss: 0.03252768889069557
Epoch: 275
Train loss: 0.031924229115247726
Test loss: 0.03245112672448158
Epoch: 276
Train loss: 0.03187728300690651
Test loss: 0.032411057502031326
Epoch: 277
Train loss: 0.03182768449187279
Test loss: 0.03235883638262749
Epoch: 278
Train loss: 0.03177323937416077
Test loss: 0.03230869024991989
Epoch: 279
Train loss: 0.031732309609651566
Test loss: 0.03225790336728096
Epoch: 280
Train loss: 0.031677111

Epoch: 377
Train loss: 0.027524227276444435
Test loss: 0.028110669925808907
Epoch: 378
Train loss: 0.027484171092510223
Test loss: 0.028072591871023178
Epoch: 379
Train loss: 0.02745509333908558
Test loss: 0.028025474399328232
Epoch: 380
Train loss: 0.027423713356256485
Test loss: 0.02800414338707924
Epoch: 381
Train loss: 0.027393681928515434
Test loss: 0.02799397148191929
Epoch: 382
Train loss: 0.027351222932338715
Test loss: 0.027928084135055542
Epoch: 383
Train loss: 0.027322599664330482
Test loss: 0.027896001935005188
Epoch: 384
Train loss: 0.027288563549518585
Test loss: 0.027845710515975952
Epoch: 385
Train loss: 0.027273930609226227
Test loss: 0.02785268984735012
Epoch: 386
Train loss: 0.02724010869860649
Test loss: 0.027829254046082497
Epoch: 387
Train loss: 0.027213364839553833
Test loss: 0.027789950370788574
Epoch: 388
Train loss: 0.027189111337065697
Test loss: 0.027790963649749756
Epoch: 389
Train loss: 0.02715689316391945
Test loss: 0.027751363813877106
Epoch: 390
Train l

...

Epoch: 345
Train loss: 0.26663658022880554
Test loss: 0.288997620344162
Epoch: 346
Train loss: 0.2666134834289551
Test loss: 0.28895989060401917
Epoch: 347
Train loss: 0.266590416431427
Test loss: 0.28892233967781067
Epoch: 348
Train loss: 0.2665673792362213
Test loss: 0.28888487815856934
Epoch: 349
Train loss: 0.2665444612503052
Test loss: 0.2888476550579071
Epoch: 350
Train loss: 0.2665216028690338
Test loss: 0.288810670375824
Epoch: 351
Train loss: 0.26649874448776245
Test loss: 0.2887738347053528
Epoch: 352
Train loss: 0.26647600531578064
Test loss: 0.2887371778488159
Epoch: 353
Train loss: 0.2664533257484436
Test loss: 0.2887006998062134
Epoch: 354
Train loss: 0.26643064618110657
Test loss: 0.28866440057754517
Epoch: 355
Train loss: 0.26640811562538147
Test loss: 0.2886282503604889
Epoch: 356
Train loss: 0.26638561487197876
Test loss: 0.28859224915504456
Epoch: 357
Train loss: 0.2663631737232208
Test loss: 0.2885564863681793
Epoch: 358
Train loss: 0.26634079217910767
Test loss: 0.

Train loss: 0.26362234354019165
Test loss: 0.28471434116363525
Epoch: 499
Train loss: 0.2636052370071411
Test loss: 0.2846930921077728
Trainer: BPTrainer (Adam_trainer)
Settings: {'method': 'AdamOptimizer', 'args': {'learning_rate': 0.005}}
Epoch: init
Train loss: 76.24555969238281
Test loss: 89.11460876464844
Epoch: 0
Train loss: 58.344974517822266
Test loss: 68.31175231933594
Epoch: 1
Train loss: 44.74446487426758
Test loss: 52.48972702026367
Epoch: 2
Train loss: 34.507015228271484
Test loss: 40.5676383972168
Epoch: 3
Train loss: 26.82599449157715
Test loss: 31.612756729125977
Epoch: 4
Train loss: 21.054109573364258
Test loss: 24.87596321105957
Epoch: 5
Train loss: 16.695707321166992
Test loss: 19.783010482788086
Epoch: 6
Train loss: 13.382044792175293
Test loss: 15.906180381774902
Epoch: 7
Train loss: 10.842714309692383
Test loss: 12.93144416809082
Epoch: 8
Train loss: 8.88032341003418
Test loss: 10.629358291625977
Epoch: 9
Train loss: 7.350700378417969
Test loss: 8.832228660583496


Epoch: 139
Train loss: 0.34403982758522034
Test loss: 0.41817936301231384
Epoch: 140
Train loss: 0.3430497944355011
Test loss: 0.4167441129684448
Epoch: 141
Train loss: 0.3420740067958832
Test loss: 0.4153282046318054
Epoch: 142
Train loss: 0.3411121666431427
Test loss: 0.4139314889907837
Epoch: 143
Train loss: 0.34016403555870056
Test loss: 0.41255369782447815
Epoch: 144
Train loss: 0.33922940492630005
Test loss: 0.41119450330734253
Epoch: 145
Train loss: 0.3383081555366516
Test loss: 0.4098537266254425
Epoch: 146
Train loss: 0.33740007877349854
Test loss: 0.40853098034858704
Epoch: 147
Train loss: 0.3365047872066498
Test loss: 0.4072258472442627
Epoch: 148
Train loss: 0.3356221318244934
Test loss: 0.4059380292892456
Epoch: 149
Train loss: 0.3347521126270294
Test loss: 0.4046672582626343
Epoch: 150
Train loss: 0.3338943421840668
Test loss: 0.40341290831565857
Epoch: 151
Train loss: 0.33304861187934875
Test loss: 0.4021748900413513
Epoch: 152
Train loss: 0.332214891910553
Test loss: 0.

Epoch: 296
Train loss: 0.27900761365890503
Test loss: 0.3164956569671631
Epoch: 297
Train loss: 0.2788723409175873
Test loss: 0.31624671816825867
Epoch: 298
Train loss: 0.2787385582923889
Test loss: 0.3160001039505005
Epoch: 299
Train loss: 0.2786061763763428
Test loss: 0.31575578451156616
Epoch: 300
Train loss: 0.27847525477409363
Test loss: 0.3155137002468109
Epoch: 301
Train loss: 0.27834582328796387
Test loss: 0.31527388095855713
Epoch: 302
Train loss: 0.27821770310401917
Test loss: 0.31503620743751526
Epoch: 303
Train loss: 0.27809104323387146
Test loss: 0.3148007392883301
Epoch: 304
Train loss: 0.2779656946659088
Test loss: 0.3145674467086792
Epoch: 305
Train loss: 0.277841717004776
Test loss: 0.3143361806869507
Epoch: 306
Train loss: 0.27771902084350586
Test loss: 0.31410712003707886
Epoch: 307
Train loss: 0.27759769558906555
Test loss: 0.31388014554977417
Epoch: 308
Train loss: 0.2774776518344879
Test loss: 0.31365519762039185
Epoch: 309
Train loss: 0.27735891938209534
Test los

Train loss: 0.26860329508781433
Test loss: 0.2957238256931305
Epoch: 441
Train loss: 0.26856690645217896
Test loss: 0.2956446409225464
Epoch: 442
Train loss: 0.2685306966304779
Test loss: 0.29556605219841003
Epoch: 443
Train loss: 0.26849472522735596
Test loss: 0.2954878807067871
Epoch: 444
Train loss: 0.2684589922428131
Test loss: 0.29541027545928955
Epoch: 445
Train loss: 0.26842355728149414
Test loss: 0.2953331768512726
Epoch: 446
Train loss: 0.2683883309364319
Test loss: 0.2952565550804138
Epoch: 447
Train loss: 0.26835331320762634
Test loss: 0.29518043994903564
Epoch: 448
Train loss: 0.2683185636997223
Test loss: 0.29510483145713806
Epoch: 449
Train loss: 0.26828405261039734
Test loss: 0.29502972960472107
Epoch: 450
Train loss: 0.2682497203350067
Test loss: 0.2949550747871399
Epoch: 451
Train loss: 0.2682156264781952
Test loss: 0.2948809564113617
Epoch: 452
Train loss: 0.2681817412376404
Test loss: 0.2948072850704193
Epoch: 453
Train loss: 0.26814812421798706
Test loss: 0.29473409